# Automação para Agendamento de Carregamento

### Imports

In [35]:
import pandas as pd
import getpass
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

### Funções

In [2]:
XPATH_usuario = '/html/body/div/div/div[1]/div[1]/div/form/div[1]/div[2]/div/input'
XPATH_senha = '/html/body/div/div/div[1]/div[1]/div/form/div[1]/div[3]/div/input'
XPATH_acessar = '/html/body/div/div/div[1]/div[1]/div/form/div[1]/div[5]/input'

# função para logar no site
def f_log():    
    # guardando usuario e senha de login vibra em variaveis
    usuario = input(prompt='usuario_vibra: ')
    senha = getpass.getpass(prompt='senha_vibra: ', stream=None)
    
    # logando no sistema vibra
    driver.find_element(By.XPATH, XPATH_usuario).send_keys(usuario) # preenche usuario
    sleep(1)
    driver.find_element(By.XPATH, XPATH_senha).send_keys(senha) # preenche senha
    sleep(1)
    driver.find_element(By.XPATH, XPATH_acessar).click() # clica acessar

In [3]:
XPATH_criar_programacao = '/html/body/div[3]/div[5]/div/div/form/div/div[1]/div/div/div[2]/div[1]/div/div[2]/div/div/div[2]/div/a[1]'
XPATH_pesquisar_remessas = '/html/body/div[2]/div[5]/div/div/div/form/div[3]/div/div/div/span[2]/input'
XPATH_placa = '/html/body/div[2]/div[5]/div/div/div/form/div[2]/span/div/div[2]/input[1]'
XPATH_buscar_placa = '/html/body/div[2]/div[5]/div/div/div/form/div[2]/span/div/div[2]/input[2]'
XPATH_lista_transport = '/html/body/div[2]/div[5]/div/div/div/form/div[3]/div[2]/div/div/select'

def criar_programacao():
    driver.find_element(By.XPATH, XPATH_criar_programacao).click() # clica em criar programação
    sleep(1)
    driver.find_element(By.XPATH, XPATH_pesquisar_remessas).click() # pesquisa remessas
    sleep(1)
    driver.find_element(By.XPATH, XPATH_placa).send_keys('PGS3G71') # preenche a placa
    sleep(1)
    driver.find_element(By.XPATH, XPATH_buscar_placa).click() # clica ok na placa
    sleep(1)
    select = Select(driver.find_element(By.XPATH, XPATH_lista_transport)) # procura a lista suspensa
    select.select_by_visible_text('06.928.937/0001-08 - PETROBARROS TRANSPORTADORA LTDA. - 0010121115') # seleciona transportador

In [4]:
XPATH_lista_horarios = '/html/body/div[2]/div[5]/div/div/div/form/div[2]/div[2]/div[2]/center/div/div[2]/div/center/select'

def busca_horario():
    # encontra o calendario
    try:
        dias = driver.find_element(By.CLASS_NAME, 'datepicker-days').find_elements(By.CLASS_NAME, 'day')
    except EC.NoSuchElementException:
        sleep(5)
        # clica em voltar
        driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[4]/div/div/div/span/a').click()
        # clica em avancar para recarregar a pagina
        sleep(2)
        driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[5]/div/div/div/span/a[2]').click()
        sleep(2)
        busca_horario()
    
    sleep(2)
    dias = driver.find_element(By.CLASS_NAME, 'datepicker-days').find_elements(By.CLASS_NAME, 'day')
    dias_ext = []
    
    # extrai os dias do calendario
    for i in range(len(dias)): 
        dias_ext.append(dias[i].text)
    
    sleep(2)
    # clica no dia desejado
    for i in range(len(dias_ext)): 
        if dias_ext[i] == dia_desejado:
            dias[dias_ext.index(dia_desejado)].click()    
    
    sleep(2)
    
    horarios_encontrados = []
    
    # o bloco try e except a seguir está sendo responsável por verificar se a lista suspensa de horarios está aparecendo.
    # se a lista suspensa estiver aparecendo o código continua. Se não, a função é chamada do início. Não é o jeito mais
    # elegante de fazer isso, mas acho que está funcionando.
    
    try:
        select_horario = Select(driver.find_element(By.XPATH,XPATH_lista_horarios))
                
    except EC.NoSuchElementException:
        sleep(5)
        # clica em voltar
        driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[4]/div/div/div/span/a').click()
        sleep(2)
        # clica em avancar para recarregar a pagina
        driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[5]/div/div/div/span/a[2]').click()
        sleep(2)
        busca_horario()
    
    sleep(2)
    select_horario = Select(driver.find_element(By.XPATH,XPATH_lista_horarios))
    
    for i in range(len(select_horario.options)): # guarda os horários disponiveis na variavel horarios_encontrados
        if int(select_horario.options[i].text[13:15]) >= horario_inicial and int(select_horario.options[i].text[13:15]) <= horario_final:
            horarios_encontrados.append(select_horario.options[i].text)
    if len(horarios_encontrados) == 0: # se não encontrar nenhum horário dentro das especificações, a função é chamada de novo
        sleep(5)
        # clica no dia atual
        driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[2]/div[2]/div[1]/center/div/div[2]/div/div/div/div[1]/table/tbody/tr[5]/td[3]').click()
        sleep(2)
        busca_horario()
        
    return horarios_encontrados

In [5]:
remetente = 'carregamentopetrobarros@gmail.com'
senha = 'cebp jrpm kcxl klev'
destinatario = ['gustavossbarros@hotmail.com', 'toninhobarrosf@hotmail.com']
assunto = 'Novo Horario de Carregamento'

def envia_email(sender_email, password, recipient, subject, body):
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = ", ".join(recipient)  # Junta os destinatários em uma string separada por vírgulas
    message['Subject'] = subject

    message.attach(MIMEText(body, 'plain'))

    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, password)
        text = message.as_string()
        server.sendmail(sender_email, recipient, text)

### Inicializando Google Chrome

In [6]:
# se essa parte do código ficar dentro de uma função o programa não consegue manter o chrome aberto

service = Service()

options = webdriver.ChromeOptions()

driver = webdriver.Chrome(service=service, options=options)

url = 'https://cn.vibraenergia.com.br/login/'

driver.get(url)

### Rodando as funções

In [7]:
f_log()

usuario_vibra: 1596189
senha_vibra: ········


In [8]:
criar_programacao()

In [9]:
dia_desejado = input(prompt='Dia Desejado: ')
horario_inicial = int(input(prompt='Horário Inicial (apenas número da hora): '))
horario_final = int(input(prompt='Horário Final (apenas número da hora): '))
horarios = busca_horario()

corpo_do_email = f'''Encontrei os seguintes horários de carregamento.
{horarios}
--------------------------------
EMAIL AUTOMÁTICO - NÃO RESPONDER
'''
envia_email(remetente, senha, destinatario, assunto, corpo_do_email)

Dia Desejado: 28
Horário Inicial (apenas número da hora): 00
Horário Final (apenas número da hora): 23


KeyboardInterrupt: 

In [39]:
# essa é a lógica para clicar no calendário

lista_dias = driver.find_elements(By.CLASS_NAME, 'day')
for i in range(len(lista_dias)):
    try:
        if lista_dias[i].text == '28':
            lista_dias[i].click()
    except EC.StaleElementReferenceException:
        pass

In [77]:
XPATH_calendario = '/html/body/div[2]/div[5]/div/div/div/form/div[3]/div[2]/div[1]/center/div/div[2]/div/div/div/div[1]/table/tbody'
string_3 = '3'

driver.find_element(By.XPATH, XPATH_calendario).find_elements(By.CLASS_NAME, 'day')[8].click()

In [ ]:
def busca_horario_v2:
    

- driver.refresh() para atualizar
- driver.find_element(By.LINK_TEXT, 'texto') para encontrar elementos baseado em texto
- driver.